1.Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [2]:
import pandas as pd

In [30]:
ratings = pd.read_csv('C:/Users/Sergei/Desktop/Python Materials/Python_8_joins/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [31]:
ltv_table = ratings.pivot_table(index = 'userId', values = 'timestamp', aggfunc = ['min', 'max']).reset_index()
ltv_table.head()

,userId,min,max
,,timestamp,timestamp
0,1,1260759108,1260759205
1,2,835355395,835356246
2,3,1298861589,1298932787
3,4,949778714,949982274
4,5,1163373044,1163375145


In [32]:
ltv_table.rename(columns = {'min':'ts_min','max':'ts_max'},inplace=True)
ltv_table.head()

,userId,ts_min,ts_max
,,timestamp,timestamp
0,1,1260759108,1260759205
1,2,835355395,835356246
2,3,1298861589,1298932787
3,4,949778714,949982274
4,5,1163373044,1163375145


In [33]:
ltv_table['ltv'] = ltv_table['ts_max'] - ltv_table['ts_min']
ltv_table.head()

,userId,ts_min,ts_max,ltv
,,timestamp,timestamp,
0,1,1260759108,1260759205,97
1,2,835355395,835356246,851
2,3,1298861589,1298932787,71198
3,4,949778714,949982274,203560
4,5,1163373044,1163375145,2101


2.Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

-таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

-аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [34]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [35]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [36]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [37]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [42]:
df = client_base.merge(rzd, how = 'outer', on = 'client_id').merge(auto, how = 'outer', on = 'client_id').merge(air, how = 'outer', on = 'client_id')
df

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [44]:
df.pivot_table(index = 'client_id', values = ['rzd_revenue', 'auto_revenue', 'air_revenue'], aggfunc = 'sum', fill_value=0)

,air_revenue,auto_revenue,rzd_revenue
client_id,,,
111,0,0,1093
112,0,0,2810
113,0,57483,10283
114,0,83,5774
115,81,912,981
116,4,4834,0
117,13,98,0
118,173,0,0


In [45]:
df.pivot_table(index = ['client_id', 'address'], values = ['rzd_revenue', 'auto_revenue', 'air_revenue'], aggfunc = 'sum', fill_value=0)

,,air_revenue,auto_revenue,rzd_revenue
client_id,address,,,
111,Комсомольская 4,0,0,1093
112,Энтузиастов 8а,0,0,2810
113,Левобережная 1а,0,57483,10283
114,Мира 14,0,83,5774
115,ЗЖБИиДК 1,81,912,981
116,Строителей 18,4,4834,0
117,Панфиловская 33,13,98,0
118,Мастеркова 4,173,0,0


3.В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

In [47]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [48]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [54]:
coordinates = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 77, 99],
        'latitude': [41.40338, 23.23423, 14.24323, 43.34566, 67.24342],
        'longitude': [2.17403, 3.13243, 4.12312, 5.12354, 6.09876],
    }
)

coordinates = coordinates[['user_id', 'latitude', 'longitude']]
coordinates

,user_id,latitude,longitude
0,11,41.40338,2.17403
1,22,23.23423,3.13243
2,55,14.24323,4.12312
3,77,43.34566,5.12354
4,99,67.24342,6.09876


In [58]:
joined = visits.merge(purchases, how='outer', on='user_id').merge(coordinates, how='outer', on='user_id')
joined

,user_id,source,category,latitude,longitude
0,11,ad,Спорт,41.40338,2.17403
1,11,ad,Спорт,41.40338,2.17403
2,11,google,Спорт,41.40338,2.17403
3,11,google,Спорт,41.40338,2.17403
4,22,yandex,Авто,23.23423,3.13243
5,55,email,Дача,14.24323,4.12312
6,77,ad,NaN,43.34566,5.12354
7,99,NaN,Авто,67.24342,6.09876


Применительно к данной ситуации просто создал бы датафрейм с координатами для пользователя. Глобально, думаю, было бы лучше привязывать координаты к визитам (у каждого визита должен быть уникальный ID)